[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RonPlusSign/AnomalySegmentation/blob/main/Project6.ipynb)

# **Anomaly Segmentation Project 6**
##*Andrea Delli, Christian Dellisanti, Giorgia Modi*

###**Dataset Preparation**

In [11]:
import sys, os

#SMIYC RA-21
if not os.path.isfile('/content/dataset_AnomalyTrack.zip'):
  !wget https://zenodo.org/record/5270237/files/dataset_AnomalyTrack.zip
if not os.path.isdir('/content/dataset_AnomalyTrack'):
  !unzip ./dataset_AnomalyTrack.zip
#SMIYC RO-21
if not os.path.isfile('/content/dataset_ObstacleTrack.zip'):
  !wget https://zenodo.org/record/5281633/files/dataset_ObstacleTrack.zip
if not os.path.isdir('/content/dataset_ObstacleTrack'):
  !unzip ./dataset_ObstacleTrack.zip

#FS L&F
if not os.path.isfile('/content/fishyscapes_lostandfound.zip'):
  !wget http://robotics.ethz.ch/~asl-datasets/Fishyscapes/fishyscapes_lostandfound.zip
if not os.path.isdir('/content/fishyscapes_lostandfound'):
  !unzip ./fishyscapes_lostandfound.zip -d fishyscapes_lostandfound

Archive:  ./fishyscapes_lostandfound.zip
  inflating: fishyscapes_lostandfound/0000_04_Maurener_Weg_8_000000_000030_labels.png  
  inflating: fishyscapes_lostandfound/0001_04_Maurener_Weg_8_000000_000090_labels.png  
  inflating: fishyscapes_lostandfound/0002_04_Maurener_Weg_8_000000_000150_labels.png  
  inflating: fishyscapes_lostandfound/0003_04_Maurener_Weg_8_000000_000180_labels.png  
  inflating: fishyscapes_lostandfound/0004_04_Maurener_Weg_8_000001_000030_labels.png  
  inflating: fishyscapes_lostandfound/0005_04_Maurener_Weg_8_000001_000090_labels.png  
  inflating: fishyscapes_lostandfound/0006_04_Maurener_Weg_8_000001_000150_labels.png  
  inflating: fishyscapes_lostandfound/0007_04_Maurener_Weg_8_000001_000180_labels.png  
  inflating: fishyscapes_lostandfound/0008_04_Maurener_Weg_8_000001_000210_labels.png  
  inflating: fishyscapes_lostandfound/0009_04_Maurener_Weg_8_000002_000060_labels.png  
  inflating: fishyscapes_lostandfound/0010_04_Maurener_Weg_8_000002_000120_labe